In [1]:
tag2id,id2tag={},{}#id和tag的映射
#tag2id={'vn':0,'nnp':1}
#id2tag={0:'vn',1:'nnp'}
word2id,id2word={},{}

In [2]:
for line in open('traindata.txt'):
    items=line.split('/')
    w,t=items[0],items[1].rstrip()
    
    if w not in word2id:
        word2id[w]=len(word2id)
        id2word[len(id2word)]=w
    if t not in tag2id:
        tag2id[t]=len(tag2id)
        id2tag[len(id2tag)]=t
m,n=len(word2id),len(tag2id)#m是词典大小，n是词性大小

In [3]:
print(m,n)

18978 54


In [4]:
#构建 pi，a，b
import numpy as np
#pi表示每个词性出现在句子中第一个词的概率是多少
pi=np.zeros(n)
#a是n*m行矩阵，表示每个单词对应每个词性的概率是多少,a[i,j]表示给定tag i，出现单词j的概率
a=np.zeros((n,m))
#b是n*n矩阵，b[i,j]表示在tag i的前提下之后是tag j的概率
b=np.zeros((n,n))

In [5]:
prev_tag=''
for line in open('traindata.txt'):
    #print(line)
    items=line.split('/')
    #print(items[0],items[1].rstrip())
    wordId,tagId=word2id[items[0]],tag2id[items[1].rstrip()]
    if prev_tag=='':#句子开始
        pi[tagId]+=1
        a[tagId,wordId]+=1
    else:#不是句子开头
        a[tagId,wordId]+=1
        b[tag2id[prev_tag],tagId]+=1
    if items[0]=='.':
        prev_tag=''
    else:
        prev_tag=items[1].rstrip()

In [6]:
#求概率
pi /= len(pi)
for i in range(n):
    a[i] /= sum(a[i])
    b[i] /= sum(b[i])

In [7]:
def log(v):
    if v==0: return np.log(v+0.00000001)
    return np.log(v)

In [8]:
def viterbi(x,pi,a,b):
    '''
    x:输入的句子
    pi:tag出现在句首的概率
    a:给定tag，每个单词出现的概率
    b:tag之间的转移方程
    '''
    x=[word2id[w] for w in x.split(' ')]
    t=len(x)
    
    dp=np.zeros((t,n))#dp[i][j]:w1..wi,假设wi的tag是第j个tag
    
    for j in range(n):
        dp[0][j]=log(pi[j])+log(a[j][x[0]])
    
    path=np.zeros((t,n),dtype=np.int)
    for i in range(1,t):#每个单词
        for j in range(n):#每个词性
            dp[i][j]=-999999
            for k in range(n):#每一个k可以到达j的所有情况
                score=dp[i-1][k]+log(b[k][j])+log(a[j][x[i]])
                if score>dp[i][j]:
                    dp[i][j]=score
                    path[i][j]=k
    
    best_tag=[0]*t
    best_tag[t-1]=np.argmax(dp[t-1])
    
    for i in range(t-2,-1,-1):
        best_tag[i]=path[i+1,best_tag[i+1]]
    
    #print(best_tag)
    for i in best_tag:
        print(id2tag[i])
    

In [9]:
x='Social Security number , passport number and details about the services provided for the payment'
viterbi(x,pi,a,b)

NNP
NNP
NN
,
NN
NN
CC
NNS
IN
DT
NNS
VBN
IN
DT
NN
